### ShorelineMonitor timeseries

In [2]:
import pandas as pd
import numpy as np

#### Annual dataset

In [7]:
# #Filter entire dataset
# path_annual = r'P:\1000545-054-globalbeaches\03_Global_shorelines_update_2020\Transect_Mapping\Version_16102018\CSV\ShorelineMonitor_1984_2021_v1.5_set1.csv'
# df_tot = pd.read_csv(path_annual)
# df_annual = df_tot[df_tot['country_name'] == 'Netherlands']
# df_annual.to_csv('shoreline_NL.csv')

# Load filtered dataset
df_annual = pd.read_csv('dash_app/assets/data/shoreline_NL.csv')

df_annual.head(2)

FileNotFoundError: [Errno 2] No such file or directory: 'dash_app/assets/data/shoreline_NL.csv'

In [7]:
#path_shorelineNL = r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\ShorelineMonitor\shoreline_NL.csv'
df_annual['changerate'] = df_annual['changerate'].round(2)
df_annual['color_cr'] = np.where(df_annual['changerate'] < 0, 'red', 'green')
df_annual['color_sandy'] = np.where(df_annual['flag_sandy'] == True, 'brown', 'lightgrey')
df_annual['flag_sandy'] = np.where(df_annual['flag_sandy'] == True, 'sand', 'other')
df_annual = df_annual.rename(columns= {'dt': 'dt_annual', 'dist': 'dist_annual'})
df_annual = df_annual.loc[:, ~df_annual.columns.str.contains('^Unnamed')]
#df.to_csv(path_shorelineNL, index = False)

df_annual.head(2)

,transect_id,country_id,continent,country_name,changerate,changerate_unc,flag_sandy,no_shorelines,dt_annual,dist_annual,...,Intersect_lat,End_lon,End_lat,coastline_idint,no_sedcomp,low_detect_shlines,err_changerate,err_timespan,color_cr,color_sandy
0,BOX_117_121_0,NLD,Europe,Netherlands,0.31,0.130099,other,31.0,"[0.0, 2.0, 6.0, 7.0, 10.0, 12.0, 13.0, 14.0, 1...","[929.9296367493945, 950.827504533188, 1247.116...",...,12.301967,-68.406639,12.308825,8637.0,1.0,1.0,1.0,1.0,green,lightgrey
1,BOX_117_121_1,NLD,Europe,Netherlands,0.07,0.124291,other,31.0,"[0.0, 2.0, 6.0, 7.0, 10.0, 12.0, 13.0, 14.0, 1...","[942.6100702962765, 943.915055522753, 1487.204...",...,12.299226,-68.410576,12.305726,8637.0,1.0,1.0,1.0,1.0,green,lightgrey


In [8]:
df_annual.to_csv('annual_timeseries.csv')

#### Monthly dataset

In [22]:
path_monthly = r'P:\11202268-hydraulic-engineering\MSc_students\Dante_van_der_Heijden\02. Data\ShorelineMonitor_monthly.csv'

df_monthly = pd.read_csv(path_monthly)
df_monthly = df_monthly.loc[:, ~df_monthly.columns.str.contains('^Unnamed')]
df_monthly = df_monthly.rename(columns= {'dt': 'dt_monthly', 'dist': 'dist_monthly', 'outliers_1': 'outliers_annual'})
df_monthly.head(2)

,transect_id,transect_origin_x,transect_origin_y,transect_end_x,transect_end_y,dt_monthly,dist_monthly,outliers,hotspot
0,BOX_117_067_110,-71.624166,10.990024,-71.607420,10.996413,"[0.0, 31.0, 62.0, 90.0, 396.0, 427.0, 455.0, 6...","[-2.857791642818455, 6.388898734846833, 6.3888...","[24, 74, 75, 76]",prograding_feature_polygon_2167
1,BOX_117_067_111,-71.625000,10.991483,-71.610989,11.002523,"[0.0, 31.0, 62.0, 90.0, 396.0, 427.0, 455.0, 6...","[-157.28827157623164, -157.28827157623164, -15...","[38, 39, 40, 83, 84]",prograding_feature_polygon_2167


In [23]:
df_merged = df_annual.merge(df_monthly, how = 'left', on= 'transect_id')
df_merged = df_merged.rename(columns= {'outliers_1': 'outliers_annual', 
                                   'outliers': 'outliers_monthly'
                                   }
                        )

In [24]:
def color_line(hotspot_name):
    if not pd.isna(hotspot_name):
        if 'retreating' in hotspot_name:
            return 'red'
        else:
            return 'green'
    else: return 'green'

df_merged['color_line'] = df_merged['hotspot'].apply(lambda x: color_line(x))

In [25]:
df_merged = df_merged[['transect_id', 'country_name', 'changerate', 'dt_monthly', 'dist_monthly', 'hotspot', 'color_line']]
df_merged = df_merged.drop_duplicates(subset='transect_id', keep=False)

df_merged.head(5)

,transect_id,country_name,changerate,dt_monthly,dist_monthly,hotspot,color_line
0,BOX_117_121_0,Netherlands,0.31,NaN,NaN,NaN,green
1,BOX_117_121_1,Netherlands,0.07,NaN,NaN,NaN,green
2,BOX_117_121_2,Netherlands,0.11,NaN,NaN,NaN,green
3,BOX_117_121_3,Netherlands,0.25,NaN,NaN,NaN,green
4,BOX_117_121_4,Netherlands,-0.06,NaN,NaN,NaN,green


In [20]:
len(df_merged), len(df_NL)

(3070, 3072)